In [7]:

import json
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


In [22]:
# 📌 3. JSON 데이터 로드
file_path = "../data/news.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 📌 4. 데이터프레임 변환
df = pd.DataFrame(data)

In [27]:
df["year"] = df["quarter"].str[:4].astype(int)  # "2006Q3" → 2006

# 📌 2006~2009년 데이터를 "2006-2009Q"로 그룹화, 나머지는 기존 분기(YYYYQ#) 유지
df["input_quarter"] = df["quarter"].apply(lambda x: "2006-2009Q" if int(x[:4]) in range(2006, 2010) else x)
df["input_year"] = df["year"].apply(lambda x: "2006-2009" if 2006 <= x <= 2009 else str(x))


In [24]:
import re

def remove_english_stopwords_dates(text):
    # 영어 단어 제거 (단어뿐만 아니라 단독 알파벳도 포함)
    text = re.sub(r'[a-zA-Z]+', '', text)

    # 날짜 패턴 제거 (예: "5日", "7月")
    text = re.sub(r'\d+[日月]', '', text)

    # 공백 제거 (텍스트가 비어있지 않게)
    text = text.strip()

    return text

# DataFrame 적용
df["tokenized_content"] = df["tokenized_content"].apply(remove_english_stopwords_dates)


In [25]:
df

,title,link,date,content,source,quarter,cleaned_content,tokenized_content
0,朝鲜试射导弹亚太骤起波澜,https://www.gmw.cn/01gmrb/2006-07/12/content_4...,2006-07-12,7月5日，朝鲜连续发射7枚导弹，引起国际社会震惊。近日，美日在安理会积极推动制裁朝鲜的议案，...,光明网,2006Q3,7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...,朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 积极 推动 制裁...
1,美朝关系开始解冻,https://www.gmw.cn/01gmrb/2007-03/08/content_5...,2007-03-08,被外界视为“破冰之旅”的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常...,光明网,2007Q1,被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 5 纽约 美国 助理 国务卿 希尔 关系...
2,美日韩间谍云集朝鲜周边空中侦察地面窃听 - 国际经济,http://intl.ce.cn/zj/200809/19/t20080919_16855...,2008-09-19,神秘，这是朝鲜留给外界的印象。 正因为神秘，一些国家的情报机构费尽心机，通过投放卫星、派遣侦...,中国经济网,2008Q3,神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...
3,美日韩紧盯朝鲜射导弹(图) - 国际经济,http://intl.ce.cn/zj/200902/11/t20090211_18171...,2009-02-11,改进型“大浦洞2号”据说可攻击美本土，甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关...,中国经济网,2009Q1,改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...,改进型 大浦洞 2 据说 攻击 美 本土 甚至 具有 搭载 核 弹头 技术 朝鲜 最近 频繁...
4,美国朝鲜问题特使称美愿与朝对话,https://world.huanqiu.com/article/9CaKrnJlEWs,2009-03-07,新华网首尔3月7日电 (记者李拯宇 干玉兰) 美国朝鲜问题特使斯蒂芬·博斯沃思7日在韩国说，...,环球网,2009Q1,新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...,新华网 首尔 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 韩国 美国 ...
...,...,...,...,...,...,...,...,...
1034,詹德斌：韩国政局变动，美国要做的不应是施压,https://hqtime.huanqiu.com/share/article/4Ky1A...,2025-01-07,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅，但防止韩国“后弹劾政局”...,环球网,2025Q1,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅但防止韩国后弹劾政局对美韩...,美国 国务卿 布林肯 抵达 韩国 。 看上去 似乎 礼仪性 道别 之 旅 防止 韩国 后...
1035,朝鲜谴责美韩军事挑衅导致地区局势恶化,http://www.xinhuanet.com/20250126/55a39b230ccf...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,新华网,2025Q1,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,新华社 首尔 电 朝中社 报道 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 近...
1036,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,光明网,2025Q1,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,中新网 电 朝中社 报道 时间 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 ...
1037,石破茂与特朗普会面，美日“小集团”谋划“大算盘” | 国际识局,http://intl.ce.cn/qqss/202502/09/t20250209_392...,2025-02-09,中新网2月9日电(记者 孟湘君)特朗普当选新一任美国总统后，意大利、以色列等国领导人先后与其...,中国经济网,2025Q1,中新网2月9日电记者 孟湘君特朗普当选新一任美国总统后意大利以色列等国领导人先后与其会面。近...,中新网 电 孟湘君 特朗普 当选 新 任 美国 总统 后 意大利 以色列 先后 会面 。...


In [32]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# 전체 데이터셋에 대해 TF-IDF를 계산 (IDF는 전체 데이터셋에 대해 계산)
def calculate_keywords_by_quarter_with_global_idf(df, quarter_col="input_quarter", content_col="tokenized_content", top_n=10):
    keywords_by_quarter = {}

    # 1. 전체 데이터셋에 대해 TF-IDF 계산 (IDF는 전체 데이터셋을 기준으로 계산)
    # 전체 데이터셋을 기준으로 IDF 값을 계산합니다.
    vectorizer = TfidfVectorizer(max_features=1000)
    vectorizer.fit(df[content_col])  # 전체 데이터셋을 기준으로 IDF 값 계산

    # 2. 분기별로 데이터를 그룹화하여 처리 (분기별로 중요 키워드 추출)
    for quarter, group in df.groupby(quarter_col):
        # 3. 해당 분기 내의 기사를 대상으로 TF-IDF 계산 (IDF는 고정값 사용)
        tfidf_matrix_quarter = vectorizer.transform(group[content_col])  # 분기 내의 TF만 계산

        # 4. 각 분기 내에서 중요한 단어들을 추출하는 함수
        def get_top_keywords_for_quarter(tfidf_matrix_quarter, top_n=10):
            # 분기 내에서 단어들의 TF-IDF 점수를 합산
            sum_tfidf = np.sum(tfidf_matrix_quarter.toarray(), axis=0)  # 분기 내의 단어들의 TF-IDF 점수 합산
            top_indices = np.argsort(sum_tfidf)[::-1][:top_n]  # 중요도 높은 순으로 상위 단어 추출
            top_words = [(vectorizer.get_feature_names_out()[i], sum_tfidf[i]) for i in top_indices]  # (단어, 중요도 점수) 저장
            return ", ".join([f"{word} ({score:.4f})" for word, score in top_words])
        # 5. 해당 분기 내에서 중요한 키워드를 추출
        keywords_by_quarter[quarter] = get_top_keywords_for_quarter(tfidf_matrix_quarter)

    return keywords_by_quarter  # 결과로 분기별 키워드를 저장한 딕셔너리 반환

# 예시로 'input_quarter' 컬럼을 기준으로 그룹화하여 각 분기별로 키워드 계산
keywords_by_quarter = calculate_keywords_by_quarter_with_global_idf(df)

# 6. 모든 분기별 결과 출력
# 각 분기별로 추출된 중요 키워드를 출력합니다.
for quarter, keywords in keywords_by_quarter.items():
    print(f"\n✅ {quarter} 분기의 TF-IDF 키워드 상위 10개:")
    print(keywords)  # 각 분기별로 중요한 키워드 출력



✅ 2006-2009Q 분기의 TF-IDF 키워드 상위 10개:
朝鲜 (4.0871), 发射 (2.2605), 导弹 (1.4539), 卫星 (1.3312), 美国 (1.1941), 安理会 (1.1397), 日本 (0.9838), 会谈 (0.9039), 韩国 (0.8200), 火箭 (0.7766)

✅ 2010Q1 분기의 TF-IDF 키워드 상위 10개:
演习 (0.4587), 参加 (0.3534), 联合 (0.3066), 军演 (0.2990), 海外 (0.2363), 军事 (0.2228), 机会 (0.2106), 美军 (0.1978), 军队 (0.1950), 韩国 (0.1683)

✅ 2010Q3 분기의 TF-IDF 키워드 상위 10개:
军演 (0.8929), 演习 (0.4647), 中国 (0.4639), 联合 (0.3880), 美国 (0.3081), 围堵 (0.2643), 最后 (0.2423), 22 (0.2176), 日本 (0.2162), 朝鲜 (0.2072)

✅ 2010Q4 분기의 TF-IDF 키워드 상위 10개:
朝鲜 (1.6255), 会谈 (1.5189), 外长 (1.2780), 日本 (1.2703), 美国 (0.9742), 报道 (0.9405), 中国 (0.8600), 半岛 (0.7367), 华盛顿 (0.6894), 军演 (0.6686)

✅ 2011Q1 분기의 TF-IDF 키워드 상위 10개:
朝鲜 (1.1499), 军演 (1.1050), 军事 (0.9017), 韩国 (0.7152), 协定 (0.6751), 日本 (0.6716), 美国 (0.6683), 签署 (0.5867), 半岛 (0.5493), 联合 (0.4833)

✅ 2011Q3 분기의 TF-IDF 키워드 상위 10개:
首脑 (0.3263), 问题 (0.2827), 李明博 (0.2402), 会谈 (0.2322), 独岛 (0.2285), 提及 (0.2164), 达成 (0.2108), 敏感 (0.1993), 共识 (0.1847), 关系 (0.1796)

✅ 2011Q4 분기의 TF-IDF 

In [46]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# 전체 데이터셋에 대해 TF-IDF를 계산 (IDF는 전체 데이터셋에 대해 계산)
def calculate_keywords_by_quarter_with_global_idf(df, quarter_col="input_quarter", content_col="tokenized_content", top_n=10):
    keywords_by_quarter = {}

    # 1. 전체 데이터셋에 대해 TF-IDF를 계산 (IDF는 전체 데이터셋에 대해 계산)
    vectorizer = TfidfVectorizer(max_features=1000)
    vectorizer.fit(df[content_col])  # 전체 데이터셋을 기준으로 IDF 값 계산
    feature_names = vectorizer.get_feature_names_out()  # 전체 데이터에서 추출된 단어 리스트
    num_features = len(feature_names)  # 단어 수 확인

    # 2. 분기별로 데이터를 그룹화하여 처리 (분기별로 중요 키워드 추출)
    results = {}  # 분기별 키워드와 문서 개수를 저장할 딕셔너리

    for quarter, group in df.groupby(quarter_col):
        total_articles_in_quarter = len(group)  # 해당 분기의 전체 기사 수

        # 3. 해당 분기 내의 기사를 대상으로 TF 계산 (IDF는 고정값 사용)
        tfidf_matrix_quarter = vectorizer.transform(group[content_col])  # 분기 내의 TF만 계산

        # 4. 각 분기 내에서 중요한 단어들을 추출하는 함수
        def get_top_keywords_for_quarter(tfidf_matrix_quarter, top_n=10):
            if tfidf_matrix_quarter.shape[0] == 0:  # 분기에 데이터가 없을 경우 예외 처리
                return []

            # 분기 내에서 단어들의 TF-IDF 점수를 합산
            sum_tfidf = np.sum(tfidf_matrix_quarter.toarray(), axis=0)  # 분기 내의 단어들의 TF-IDF 점수 합산
            sum_tfidf_normalized = sum_tfidf / total_articles_in_quarter  # 문서 수로 정규화

            # top_n이 실제 단어 수보다 크지 않도록 제한
            actual_top_n = min(top_n, num_features, len(sum_tfidf_normalized))
            if actual_top_n == 0:
                return []

            # 중요도 높은 순으로 상위 단어 추출
            top_indices = np.argsort(sum_tfidf_normalized)[::-1][:actual_top_n]

            # TF 점수와 IDF 점수를 계산
            tf_scores = [sum_tfidf[i] for i in top_indices]
            idf_scores = [vectorizer.idf_[i] for i in top_indices]

            # 예외 처리: 인덱스가 범위를 초과하지 않도록
            top_words = []
            for i in top_indices:
                if i < num_features:  # 인덱스 범위 초과 방지
                    top_words.append((feature_names[i], tf_scores[top_indices.tolist().index(i)], idf_scores[top_indices.tolist().index(i)], sum_tfidf_normalized[i]))

            return top_words  # (단어, TF 점수, IDF 점수, TF-IDF 점수) 리스트 반환

        # 5. 해당 분기 내에서 중요한 키워드를 추출
        keywords_by_quarter = get_top_keywords_for_quarter(tfidf_matrix_quarter)

        # 6. 결과 저장 (분기명, 문서 개수, 키워드 목록)
        results[quarter] = {"document_count": total_articles_in_quarter, "keywords": keywords_by_quarter}

    return results  # 결과로 분기별 키워드와 문서 개수를 저장한 딕셔너리 반환

# 예시로 'input_quarter' 컬럼을 기준으로 그룹화하여 각 분기별로 키워드 계산
results = calculate_keywords_by_quarter_with_global_idf(df)

# 7. 모든 분기별 결과 출력
# 각 분기별로 추출된 중요 키워드와 TF, IDF 점수, 문서 개수를 출력합니다.
for quarter, data in results.items():
    document_count = data["document_count"]
    keywords = data["keywords"]

    print(f"\n <{quarter} 분기> : {document_count}개")
    print("단어", "TF 점수", "IDF 점수", "TF-IDF 점수")  # 제목 출력
    for word, tf_score, idf_score, tfidf_score in keywords:
        print(f"{word} ({tf_score:.4f}) ({idf_score:.4f}) ({tfidf_score:.4f})")  # 각 분기별 키워드 및 점수 출력



 <2006-2009Q 분기> : 14개
단어 TF 점수 IDF 점수 TF-IDF 점수
朝鲜 (4.0871) (1.3608) (0.2919)
发射 (2.2605) (2.5089) (0.1615)
导弹 (1.4539) (1.8092) (0.1038)
卫星 (1.3312) (3.1934) (0.0951)
美国 (1.1941) (1.0635) (0.0853)
安理会 (1.1397) (3.2741) (0.0814)
日本 (0.9838) (1.1535) (0.0703)
会谈 (0.9039) (1.9406) (0.0646)
韩国 (0.8200) (1.1435) (0.0586)
火箭 (0.7766) (3.8865) (0.0555)

 <2010Q1 분기> : 1개
단어 TF 점수 IDF 점수 TF-IDF 점수
演习 (0.4587) (2.2267) (0.4587)
参加 (0.3534) (2.4018) (0.3534)
联合 (0.3066) (1.7364) (0.3066)
军演 (0.2990) (2.5398) (0.2990)
海外 (0.2363) (4.0152) (0.2363)
军事 (0.2228) (1.5140) (0.2228)
机会 (0.2106) (3.5775) (0.2106)
美军 (0.1978) (2.2399) (0.1978)
军队 (0.1950) (3.3122) (0.1950)
韩国 (0.1683) (1.1435) (0.1683)

 <2010Q3 분기> : 2개
단어 TF 점수 IDF 점수 TF-IDF 점수
军演 (0.8929) (2.5398) (0.4464)
演习 (0.4647) (2.2267) (0.2323)
中国 (0.4639) (1.6262) (0.2319)
联合 (0.3880) (1.7364) (0.1940)
美国 (0.3081) (1.0635) (0.1541)
围堵 (0.2643) (4.0152) (0.1322)
最后 (0.2423) (3.2108) (0.1211)
22 (0.2176) (4.0152) (0.1088)
日本 (0.2162) (1.1535

In [30]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
#
# # 전체 데이터셋에 대해 TF-IDF를 계산 (IDF는 전체 데이터셋에 대해 계산)
# def calculate_keywords_by_quarter_with_article_ratio(df, quarter_col="input_quarter", content_col="tokenized_content", top_n=10):
#     keywords_by_quarter = {}
#
#     # 1. NaN 또는 None 값을 빈 문자열로 변환하여 TfidfVectorizer가 처리할 수 있도록 함
#     df[content_col] = df[content_col].fillna("")  # NaN 또는 None 값을 빈 문자열로 처리
#
#     # 2. 전체 데이터셋에 대해 TF-IDF 계산 (IDF는 전체 데이터셋을 기준으로 계산)
#     vectorizer = TfidfVectorizer(max_features=1000)
#     tfidf_matrix = vectorizer.fit_transform(df[content_col])  # 전체 데이터셋을 기준으로 TF-IDF 계산
#     feature_names = vectorizer.get_feature_names_out()  # 계산된 단어 리스트
#
#     # 3. 분기별로 데이터를 그룹화하여 처리 (분기별로 중요 키워드 추출)
#     for quarter, group in df.groupby(quarter_col):
#         # 4. 각 분기 내에서 등장한 단어의 등장한 기사 수를 계산
#         # 5. 각 분기의 기사 수
#         total_articles_in_quarter = len(group)
#
#         # 6. 각 단어가 등장한 기사 수 계산
#         term_frequencies = np.zeros(len(feature_names))  # 각 단어가 등장한 기사 수를 저장
#
#         for idx, row in group.iterrows():
#             # 해당 기사에서 등장한 단어를 찾음
#             words_in_article = set(row[content_col].split())  # 해당 기사의 단어 리스트 (set으로 중복 제거)
#             for word in words_in_article:
#                 if word in feature_names:
#                     word_idx = feature_names.tolist().index(word)  # 해당 단어의 인덱스를 찾음
#                     term_frequencies[word_idx] += 1  # 해당 단어가 등장한 기사 수를 증가시킴
#
#         # 7. 등장한 기사 수 / 전체 기사 수로 비율 계산 (단어의 중요도 비율)
#         term_frequencies_ratio = term_frequencies / total_articles_in_quarter
#
#         # 8. 비율 계산: 각 단어의 비율 (전체 문서 내에서의 단어의 중요도에 대한 상대적인 비율)
#         top_indices = np.argsort(term_frequencies_ratio)[::-1][:top_n]  # 중요도 높은 순으로 상위 단어 추출
#         top_words = [(feature_names[i], term_frequencies_ratio[i]) for i in top_indices]  # (단어, 중요도 점수) 저장
#
#         # 9. 해당 분기 내에서 중요한 키워드를 추출
#         keywords_by_quarter[quarter] = ", ".join([f"{word} ({score:.4f})" for word, score in top_words])
#
#     return keywords_by_quarter  # 결과로 분기별 키워드를 저장한 딕셔너리 반환
#
# # 예시로 'input_quarter' 컬럼을 기준으로 그룹화하여 각 분기별로 키워드 계산
# keywords_by_quarter = calculate_keywords_by_quarter_with_article_ratio(df)
#
# # 10. 모든 분기별 결과 출력
# # 각 분기별로 추출된 중요 키워드를 출력합니다.
# for quarter, keywords in keywords_by_quarter.items():
#     print(f"\n✅ {quarter} 분기의 TF-IDF 키워드 상위 10개:")
#     print(keywords)  # 각 분기별로 중요한 키워드 출력



✅ 2006-2009Q 분기의 TF-IDF 키워드 상위 10개:
韩国 (0.8571), 美国 (0.8571), 朝鲜 (0.8571), 日本 (0.7857), 报道 (0.7143), 问题 (0.6429), 紧张 (0.5714), 进行 (0.5714), 发射 (0.5714), 安全 (0.5000)

✅ 2010Q1 분기의 TF-IDF 키워드 상위 10개:
方面 (1.0000), 韩国 (1.0000), 韩军 (1.0000), 积极 (1.0000), 进行 (1.0000), 环球网 (1.0000), 正在 (1.0000), 方案 (1.0000), 海外 (1.0000), 报道 (1.0000)

✅ 2010Q3 분기의 TF-IDF 키워드 상위 10개:
22 (1.0000), 防卫 (1.0000), 韩国 (1.0000), 之后 (1.0000), 参加 (1.0000), 反潜 (1.0000), 军事 (1.0000), 军演 (1.0000), 出现 (1.0000), 关系 (1.0000)

✅ 2010Q4 분기의 TF-IDF 키워드 상위 10개:
朝鲜 (1.0000), 报道 (1.0000), 日本 (1.0000), 美国 (1.0000), 会谈 (1.0000), 进行 (0.7500), 问题 (0.7500), 环球网 (0.7500), 外长 (0.7500), 局势 (0.7500)

✅ 2011Q1 분기의 TF-IDF 키워드 상위 10개:
韩国 (1.0000), 军事 (1.0000), 美国 (1.0000), 朝鲜 (1.0000), 半岛 (1.0000), 同盟 (0.8333), 进行 (0.8333), 规模 (0.8333), 日本 (0.8333), 局势 (0.8333)

✅ 2011Q3 분기의 TF-IDF 키워드 상위 10개:
访问 (1.0000), 方面 (1.0000), 首脑 (1.0000), 方案 (1.0000), 赔偿 (1.0000), 竹岛 (1.0000), 有关 (1.0000), 美国 (1.0000), 相互 (1.0000), 首相 (1.0000)

✅ 2011Q4 분기의 TF-IDF 키